In [1]:
setwd("/hpc/hub_oudenaarden/agiladi/data/tuft_release/")

In [2]:
library(metacell)
library(Hmisc)
library(RColorBrewer)
source("release_code/metacell_functions.r")
scdb_init("saved_work", force=T)

Warning message:
“package ‘Hmisc’ was built under R version 3.6.3”Loading required package: lattice
Warning message:
“package ‘lattice’ was built under R version 3.6.3”Loading required package: survival
Warning message:
“package ‘survival’ was built under R version 3.6.3”Loading required package: Formula
Warning message:
“package ‘Formula’ was built under R version 3.6.3”Loading required package: ggplot2
Warning message:
“package ‘ggplot2’ was built under R version 3.6.3”
Attaching package: ‘Hmisc’

The following objects are masked from ‘package:base’:

    format.pval, units

Loading required package: reshape2
Warning message:
“package ‘reshape2’ was built under R version 3.6.3”Loading required package: plyr

Attaching package: ‘plyr’

The following objects are masked from ‘package:Hmisc’:

    is.discrete, summarize

Loading required package: plotrix


ERROR: Error in scdb_init("saved_work", force = T): cannot initialize db to non existing directory base_dir


In [ ]:
id = "10x"

outdir = paste0("release_figures/fig3_10x/")
dir.create(outdir)
scfigs_init(outdir)

In [ ]:

#id = "ileum_organoids"
suffix = ""; id_f = paste0(id, suffix)
sc_2d = scdb_mc2d(id); sc_cl = scdb_mc(paste0(id, suffix)); sc_mat = scdb_mat(id)

cells = names(sc_cl@mc)
cell_stats = sc_mat@cell_metadata[cells,]
colnames(cell_stats) = gsub(" ", ".", colnames(cell_stats))
colnames(cell_stats) = gsub("\\(|\\)", "_", colnames(cell_stats))
fp = sc_cl@mc_fp
lfp = log2(sc_cl@mc_fp)


In [ ]:
color_scheme = sc_cl@color_key
color2name = as.vector(color_scheme$group); names(color2name) = color_scheme$color
name2color = as.vector(color_scheme$color); names(name2color) = color_scheme$group

sc_names = color2name[ sc_cl@colors[ sc_cl@mc]]; names(sc_names) = names(sc_cl@mc)
annotations = as.matrix(read.delim(paste0("release_import/figure3_10x/lin_ord_", id, ".txt"), stringsAsFactor=F, h=F))[,1]
lin_ord = annotations #intersect(annotations, names(table(sc_names)))
#lin_ord = names(table(sc_names))

In [ ]:
bad_cells = names(which(is.na(sc_names))) 
good_cells = setdiff(names(sc_cl@mc), bad_cells)
good_mc = names(table(sc_cl@mc[ good_cells]))
bad_mc = setdiff(names(table(sc_cl@mc)), good_mc)
clust_ord = setdiff(order(factor(color2name[ sc_cl@colors], levels = lin_ord)), bad_mc)

message( length(good_cells), " good cells!")

In [ ]:
comb = paste0(cell_stats$seeder_cell, "@", cell_stats$medium); names(comb) = rownames(cell_stats)

In [ ]:
p = function() {
    plot(sc_2d@sc_x[ good_cells], sc_2d@sc_y[ good_cells], pch = 21, bg = sc_cl@colors[ sc_cl@mc[ good_cells]], axes = F,
        xlab = "", ylab = "")
    axis(1); axis(2, las = 2)
    
}

p()
pdf(paste0(outdir, "/Fig3b.pdf"), useDingbats = F)
p()
dev.off()

#good_pops = setdiff(lin_ord, bad_pops)
plot.empty()
legend("topleft", lin_ord, pch = 21, pt.bg = name2color[ lin_ord])


In [ ]:

umis = read_large_umis(id, cells = good_cells)
umis_n = sweep(umis, 2, colSums(umis), "/") * 1000


In [ ]:
plot_feature_maps = function(i, genes, nr = 2, nc = 3, rfac = 5) {
    cb = 1 / nc
    rb = 1 / nr
    locs = cbind(rep(seq_len(nc) - 1, nr), rep(rev(seq_len(nr)) - 1, each = nc))
    gene = genes[i]
    all_cells = names(sort(umis_n[gene, good_cells]))
    vals = umis_n[gene, good_cells]
    all_cells = names(sort(vals))
    pos_vals = log(1 + 1 * vals[ vals > 0])
    pos_vals = pmin(pos_vals, quantile(pos_vals, 0.99))
    val_n = rep(1, length(all_cells)); names(val_n) = all_cells
    val_n[ names(pos_vals)] = round(100 * pos_vals / max(pos_vals)) + 1
    grad = colorRampPalette(c("gray90", "gray90", brewer.pal(3, "Reds")))(101)
    cell_ord = names(val_n) #sort(val_n))
    r = locs[i,2]; c = locs[i,1]
    par(fig = c(c / nc, c / nc + cb * 0.85, r / nr, r / nr + rb * 0.85), mar = rep(0.5,4), new = (i > 1))
    df = cbind(x = sc_2d@sc_x[ all_cells], y = sc_2d@sc_y[ all_cells], col = grad[ val_n[ all_cells]], 
               rx = round(sc_2d@sc_x[ all_cells] / rfac), ry = round(sc_2d@sc_y[ all_cells] / rfac))
    df = df[ !duplicated(df[, c("col", "rx", "ry")]),]
    dim(df)
    plot(df[,"x"], df[,"y"], pch = 20, cex = 0.7, col = df[,"col"], axes = F, xlab = "", ylab = "", main = gene)
    par (fig = c(c / nc + cb * 0.9, c / nc + cb * 1, r / nr + rb * 0.3, r / nr + rb * 0.6), new=T)
    ax = seq(0, max(pos_vals), length.out = length(grad))
    image(y = ax, t(seq_along(grad)), axes = F, col = grad)
    box(); axis(2, at = quantile(ax, c(0,1)), labels = round(quantile(ax, c(0,1)),3), las = 2)
}


In [ ]:
genes = c("POU2F3", "KIT", "AVIL",
          "LGR5", "OLFM4", "ITLN1", "FCGBP", "MUC2",
          "CHGA", "NEUROG3", "CHGB")


pdf(paste0(outdir, "/ExtFig6j_right.pdf"), height=15, width=20, useDingbats=F)
invisible(sapply(seq_along(genes), plot_feature_maps, genes, 3, 4))
dev.off()

In [ ]:
m = t(apply(umis_n[genes, good_cells], 1, tapply, sc_names[ good_cells], mean))
m = m[, intersect(lin_ord, names(which(table(sc_names[ good_cells]) > 20)))]
m = m[ order(max.col(m)),]
IM = log(m + 0.02)
exp_freq = t(apply(umis[rownames(m), good_cells] > 0, 1, tapply, sc_names[ good_cells], mean)) * 100
exp_freq = exp_freq[ ,colnames(m)]

dimnames(exp_freq) = dimnames(IM)
grad = colorRampPalette(c("white", brewer.pal(9, "YlOrBr")))(101)

p = function() {
    par(mar = c(3,8,1,3))
    matplot.2(t(IM), t(exp_freq), grad=grad, cex_lim = seq(0,100, by = 25), zlim = c(-3.91202, 1.8961103))
}

p()
pdf(paste0(outdir, "/ExtFig6j_left.pdf"), height=10, width=15, useDingbats = F)
p()
dev.off()

In [ ]:

comb = with(cell_stats, paste0(medium, "@", seeder_cell)); names(comb) = rownames(cell_stats)
comb_cols = c("#D3DEE0", "antiquewhite", "dodgerblue", "darkorange2"); names(comb_cols) = names(table(comb))

In [ ]:
cols = c("darkorange2", "antiquewhite", "gray", "#D3DEE0", "#897155", "#7030A0")
names(cols) = c("stimulated", "naive", "KO", "epithel", "fetal", "traced")

In [ ]:
p = function() {
    plot(sc_2d@sc_x[ good_cells], sc_2d@sc_y[ good_cells], pch = 21, lwd = 0, bg = comb_cols[ comb[ good_cells]], axes = F,
        xlab = "", ylab = "")
    axis(1); axis(2, las = 2)

}

p()
pdf(paste0(outdir, "/Fig3c.pdf"), useDingbats = F)
p()
dev.off()


In [ ]:
sample_dist = table(comb[ good_cells], factor(sc_names[ good_cells], levels = lin_ord))

p = function() {
    par(mfrow = c(2,4), mar = c(1, 5, 3, 1))
    for(c in lin_ord) {
        bconf = 100 * binconf(sample_dist[,c], rowSums(sample_dist)); rownames(bconf) = rownames(sample_dist) 
        par(mar = c(1,7,3,1))
        coords = barplot(bconf[,"PointEst"], las = 2, main = c, ylab = "Cell frequency", ylim = c(0, 1.05 * max(bconf[,"Upper"])), names.arg = rep("", nrow(sample_dist)),
                        col = comb_cols[ rownames(sample_dist)])
        segments(coords, bconf[,"Lower"], y1 = bconf[,"Upper"])
        segments(coords - 0.2, bconf[,"Lower"], coords + 0.2)
        segments(coords - 0.2,
                 bconf[,"Upper"], coords + 0.2)
    }
}

p()
pdf(paste0(outdir, "/ExtFig6k.pdf"), height = 6, width=12)
p()
dev.off()